In [51]:
# This code creates a (k, lambda, eta, beta)- base synopsis generator that is (epsilon,delta)-DP 

import numpy as np
import math
import sympy as sp
from itertools import combinations
import pandas as pd
import time
import copy


lower_bound = -1 # data lower bound
upper_bound = 1  # data upper bound
data_precision = 2 # data precision

n = 40
num_points = 2*n # number of data points
m = num_points*math.ceil(math.log2(((upper_bound - lower_bound)/10**(-data_precision)) +1)) # size of the domain universe

delta =  0.1 # DP parameter
eta = 0.3 # edge for boosting
beta = 0.3 # failure probability of the base synopsis
k = math.ceil(2*((math.log(2/beta)+m)/(1-2*eta))) # number of query sample as demanded by Lemma 6.5 
# Assume coefficient \in (0,1], changing one x_i can at most change 1*[(1+x_j)^2 - (-1+x_j)^2] = 4x_j <= 4.

# consider the following query:
# q(X) = c/n (sum(x_i^2))
rho = 1/num_points # l_1 sensitivity 




Llambda = 0.01 # accuracy parameter lambda
epsilon = (math.log(1/beta)*rho*math.sqrt(k*math.log(1/delta)))/Llambda
epsilon = math.trunc(epsilon*100)/100+10**(-data_precision) # round epsilon up to 2 decimal points 

# to study just one query, set k=1
# k = 2


In [52]:
# query function


In [53]:
import random 
# decide on a set of real data 
real_X = np.random.choice(np.arange(-1, 1.01, 0.01), num_points, replace=False)
sum_real_X_squared = np.sum(np.square(real_X))

# initialize the synopsis to be some arbirary set of data, say from the standard normal
# for verification purposes, if fake_X = real_X, the initial error should be the same as the added laplace noise
# fake_X = real_X 
fake_X = np.random.randn(num_points)
# fake_X = np.ones(num_points)*(-0.9)


fake_X_normalizer = np.max(np.abs(fake_X))
for idx, i in enumerate(fake_X):
    if np.abs(i)>=1 :
        fake_X[idx] = i/fake_X_normalizer
# fake_X = np.random.uniform(low=lower_bound, high=upper_bound, size=num_points)
fake_X_copy = copy.copy(fake_X) # save a copy of fake_X
sum_fake_X_squared = np.sum(np.square(fake_X_copy))



all_coeff = np.round(1-np.linspace(0,100000,100000, endpoint=False)/100000,5)
sampled_queries = np.array(random.sample(list(all_coeff),k))




#### BOOSTING LOOP STARTS ####

# initialize all-zero arrays to store noiselss query output, noisy output, and laplace noise  
real_output = np.zeros(k)
real_data_noisy_output = np.zeros(k)
lap_noise = np.zeros(k)
fake_output = np.zeros(k)
error = np.zeros(k) # store |q(X) - noisy_output| for each q



# for each query, compute its real output, noisy output, and initial error
for index, item in enumerate(sampled_queries):
    
    # real output 
    real_output[index] = item*sum_real_X_squared/num_points # quadratic query
    # real_output[index] = item*np.sum(real_X)/num_points # linear query

    
    # # compute noisy output on the real data
    # k_actual = math.ceil(2*((math.log(2/beta)+m)/(1-2*eta))) # number of query sample as demanded by Lemma 6.5 
    # lap_noise[index] = np.random.laplace(loc=0.0, scale=rho*(2*math.sqrt(2*k*math.log(1/delta))/epsilon), size=None) 
    # artificially tune the lap noise down by a half
    lap_noise[index] = 0.2*np.random.laplace(loc=0.0, scale=rho*(2*math.sqrt(2*k*math.log(1/delta))/epsilon), size=None)
    # lap_noise[index] = 0
    real_data_noisy_output[index] = real_output[index] + lap_noise[index]

    # # compute query output on fake data 
    fake_output[index] = item*sum_fake_X_squared/num_points # quadratic query
    # fake_output[index] = item*np.sum(fake_X)/num_points # linear query

    # # calculate initial error
    # # notice that this is |q(X) - real_data_noisy_output|
    error[index] = abs(fake_output[index]-real_data_noisy_output[index])



In [50]:
# import matplotlib.pyplot as plt

# all_coeff = np.round(1-np.linspace(0,100000,100000, endpoint=False)/100000,5)
# sampled_queries = np.array(random.sample(list(all_coeff),k))


# loss_gradient = np.zeros(201)
# query_grad_wrt_x = np.zeros(201)
# for idx,item in enumerate(np.linspace(-1, 1, 201)):
#     xi = item
#     # calculate the dq_j/dx_i for all j = 1, ..., k
#     query_grad_wrt_xi_array = 2*xi*sampled_queries/num_points
#     query_grad_wrt_x[idx] = query_grad_wrt_xi_array[0]
#     loss_gradient[idx] = 2/Llambda**2*np.sum((sampled_queries/num_points*sum_fake_X_squared - real_data_noisy_output)*query_grad_wrt_xi_array)

# # Create a scatter plot
# plt.scatter(np.linspace(-1,1,201), loss_gradient,color='blue')
# plt.scatter(np.linspace(-1,1,201),query_grad_wrt_x,color ='red')
# # plt.title('Scatter Plot of y = x^2')
# # plt.xlabel('x')
# # plt.ylabel('y')
# plt.legend()
# plt.grid(True)
# plt.show()


sum(lap_noise>Llambda/2)


557

In [54]:

#### COORDINATE DESCENT LOOP STARTS HERE ###
#### In this loop, we do coordinate descent, NOT multivariate Newton's method ####

# initialize number of coordinate descent iterations = 0
num_iter_descent = 0

x_coord_changed = np.full(num_points, False, dtype=bool)
x_coord_changed = {i: x_coord_changed[i] for i in range(len(x_coord_changed))}


# while we don't have |q(X) - noisy_output|<lambda/2 for all q, continue coordinate descent 
while not np.all(error < Llambda/2):
    
    # calculate the current total loss, which includes two copies of loss for each query function
    total_loss_plus = np.sum(np.exp((fake_output-real_data_noisy_output)/Llambda-1))
    total_loss_minus = np.sum(np.exp((real_data_noisy_output-fake_output)/Llambda-1))
    total_loss = total_loss_plus + total_loss_minus 

    ### compute the partial derivative of the loss function with respect to each coordinate 
    loss_gradient = np.zeros(num_points) # initialize the partial derivative of each coordinate to be zero
    
    # fix xi    
    for i in range(num_points):
        xi = fake_X[i]

        # calculate the dq_j/dx_i for all j = 1, ..., k
        query_grad_wrt_xi_array = 2*xi*sampled_queries/num_points
        loss_gradient_plus = np.sum(2*sampled_queries*xi*np.exp((fake_output-real_data_noisy_output)/Llambda - 1)/(Llambda*num_points))
        loss_gradient_minus = np.sum(-2*sampled_queries*xi*np.exp(-(fake_output-real_data_noisy_output)/Llambda - 1)/(Llambda*num_points))
        loss_gradient[i] = loss_gradient_plus + loss_gradient_minus

    # create a dictionary for loss_gradient
    loss_gradient_dict = {i: loss_gradient[i] for i in range(len(loss_gradient))}


    # find the coordinate with the max absolute value part_derivative 
    filtered_loss_grad_dict = {k: v for k, v in loss_gradient_dict.items() if not x_coord_changed[k]}
    x_coord_descent = max(filtered_loss_grad_dict, key=lambda k: abs(filtered_loss_grad_dict[k]))

    
        

    # update x value at the coordinate x_coord_descent
    # First, we calculate the 2nd derivative wrt x_coord_descent
    xi = fake_X[x_coord_descent] 
    # quadratic query

    loss_hessian_wrt_chosen_xi_plus = np.sum((2*sampled_queries)/(Llambda*num_points)*np.exp((fake_output-real_data_noisy_output)/Llambda -1)*(1+2*sampled_queries*xi**2/(Llambda*num_points)))
    loss_hessian_wrt_chosen_xi_minus = np.sum(-(2*sampled_queries)/(Llambda*num_points)*np.exp(-(fake_output-real_data_noisy_output)/Llambda -1)*(1-2*sampled_queries*xi**2/(Llambda*num_points)))
    loss_hessian_wrt_chosen_x_coord = loss_hessian_wrt_chosen_xi_plus + loss_hessian_wrt_chosen_xi_minus
    # loss_hessian_wrt_chosen_x_coord = sampled_queries/num_points # linear queries

    
    ### Backtracking Line Search Sub-routine ###
    # Backtracking line search 
    # initialize a step size, constants alpha1 and alpha2
    t = 1
    alpha1 = 0.05 # usually 0 < alpha1 < 0.5
    alpha2 = 0.5 # usually 0 < alpha2 < 1

    # initialize AG_condition = 0
    AG_condition = 0

    # make a copy of the curent fake_X
    fake_X_copy_linesearch = copy.copy(fake_X)

    while AG_condition == 0: 

        # calculate f(x+t*Delta x)
        fake_X_copy_linesearch[x_coord_descent] = xi - t*loss_gradient[x_coord_descent]/loss_hessian_wrt_chosen_x_coord
        # update fake_output 
        sum_fake_X_squared_linesearch = np.sum(np.square(fake_X_copy_linesearch))
        fake_output_linesearch = sampled_queries*sum_fake_X_squared_linesearch /num_points
        # fake_output_linesearch = sampled_queries*np.sum(fake_X_copy_linesearch) /num_points
        # loss_current_stepsize = np.sum(np.square(fake_output_linesearch-real_data_noisy_output))/Llambda
        loss_current_stepsize_plus = np.sum(np.exp((fake_output_linesearch-real_data_noisy_output)/Llambda-1))
        loss_current_stepsize_minus = np.sum(np.exp((real_data_noisy_output-fake_output_linesearch)/Llambda-1))
        loss_current_stepsize = loss_current_stepsize_plus + loss_current_stepsize_minus 

        print('loss_current', loss_current_stepsize)

        # calculate f(x)+alpha1*t*grad_f*Delta x
        loss_damped_stepsize = total_loss + alpha1*t*loss_gradient[x_coord_descent]*(- loss_gradient[x_coord_descent]/loss_hessian_wrt_chosen_x_coord)
        print('loss_dampened', loss_damped_stepsize)
        
        # if loss_current_stepsize > loss_damped_stepsize or -1>xi - t*loss_gradient[x_coord_descent]/loss_hessian_wrt_chosen_x_coord or xi - t*loss_gradient[x_coord_descent]/loss_hessian_wrt_chosen_x_coord>1:
        if loss_current_stepsize > loss_damped_stepsize:
            t = t* alpha2
            
            
            print("t = ", t)

        else: 
            AG_condition = 1 
            print("t = ", t)
    ############



    # update fake_X at coordinate = x_coord_descent using 2nd order Newton's method 
    xi_update = - t*loss_gradient[x_coord_descent]/loss_hessian_wrt_chosen_x_coord
    fake_X[x_coord_descent] = xi + xi_update

    if fake_X[x_coord_descent] <= -1 or fake_X[x_coord_descent] >= 1:
        x_coord_changed[x_coord_descent] = True 
        
    # if xi+xi_update <= 1 and xi+xi_update >= -1:
    #     fake_X[x_coord_descent] = xi + xi_update
    # # if the proposed update will overshoot, we skip this coordinate 
    # else:
    #     x_coord_changed[x_coord_descent] = True
    

    # update fake_output 
    sum_fake_X_squared = np.sum(np.square(fake_X))
    fake_output = sampled_queries*sum_fake_X_squared /num_points # quadratic query 
    # fake_output = sampled_queries*np.sum(fake_X)/num_points # linear query


    # update error
    error = abs(fake_output-real_data_noisy_output)

    # update total loss
    total_loss_plus = np.sum(np.exp((fake_output-real_data_noisy_output)/Llambda-1))
    total_loss_minus = np.sum(np.exp((real_data_noisy_output-fake_output)/Llambda-1))
    total_loss = total_loss_plus + total_loss_minus 

    # print current progress
    print(f"#iter {num_iter_descent} x_co={x_coord_descent}, 1st={loss_gradient[x_coord_descent]}, 2nd={loss_hessian_wrt_chosen_x_coord}, # queries above err={sum(error>Llambda/2)}, xi={xi} fake x={fake_X[x_coord_descent]}")
    print('Total loss = ', total_loss)
    
    num_iter_descent += 1 




loss_current 3216.984571534762
loss_dampened 9437.434461671033
t =  1
#iter 0 x_co=7, 1st=-17174.326513056996, 2nd=18005.390597819354, # queries above err=1379, xi=1.0 fake x=1.953843595880502
Total loss =  3216.984571534762
loss_current 3086.016425760138
loss_dampened 3205.687253142565
t =  1
#iter 1 x_co=6, 1st=-1341.6289229876304, 2nd=7966.35141415591, # queries above err=1117, xi=-0.9680983256109373 fake x=-0.7996863571653067
Total loss =  3086.016425760138
loss_current 3074.5709647625144
loss_dampened 3084.9393390394594
t =  1
#iter 2 x_co=35, 1st=359.08173188292244, 2nd=5985.57607741937, # queries above err=1119, xi=0.9283432787172552 fake x=0.8683521053968524
Total loss =  3074.5709647625144
loss_current 3074.456427301245
loss_dampened 3074.559590487762
t =  1
#iter 3 x_co=78, 1st=-35.42646413885791, 2nd=5516.986307604765, # queries above err=1119, xi=-0.9220563244174195 fake x=-0.915634980922326
Total loss =  3074.456427301245
loss_current 3074.4564145692284
loss_dampened 3074.

KeyboardInterrupt: 

In [36]:
sum

3210